In [123]:
class YClass( object ):
    pass

args = YClass()
setattr(args,'config_file','/Users/matentzn/knocean/data-harmonization/src/mapping-suggest/mapping-suggest-config.yml')
setattr(args,'tsv_path','/Users/matentzn/knocean/data-harmonization/build/intermediate/koges-xrefs-sparql.csv')
setattr(args,'template_path','/Users/matentzn/knocean/data-harmonization/templates/koges.tsv')
setattr(args,'file_simplestring','/Users/matentzn/knocean/data-harmonization/data/ihcc-mapping-suggestions-zooma-simplestring.tsv')
setattr(args,'tsv_out_path','/Users/matentzn/knocean/data-harmonization/templates/_koges_nlp.tsv')
setattr(args,'gecko_path','/Users/matentzn/knocean/data-harmonization/build/intermediate/gecko-xrefs-sparql.csv')


In [124]:
import os
import pandas as pd
import re
from datetime import datetime
import csv
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [125]:
obo_purl="http://purl.obolibrary.org/obo/"
def obo_id_to_iri(obo_id):
    if obo_id and isinstance(obo_id,str):
        if obo_id.startswith(obo_purl):
            return obo_id
        elif ":" in obo_id:
            return obo_purl+obo_id.replace(":","_")
    return obo_id
    
def compute_string_variants(label):
    label_alpha = re.sub('[^0-9a-zA-Z ]+', ' ', label)
    label_alpha = re.sub('\s+', ' ', label_alpha)
    label_split_numerics_alpha = re.sub('(?<=\d)(?!\d)|(?<!\d)(?=\d)', ' ', label_alpha)
    label_split_numerics_alpha_camel = re.sub("([a-z])([A-Z])","\g<1> \g<2>",label_split_numerics_alpha)
    label_split_numerics_alpha = re.sub('\s+', ' ', label_split_numerics_alpha)
    label_split_numerics_alpha_camel = re.sub('\s+', ' ', label_split_numerics_alpha_camel)
    #label_split_numerics = re.sub('(?<=\d)(?!\d)|(?<!\d)(?=\d)', ' ', label)
    #label_split_numerics = re.sub('\s+', ' ', label_split_numerics)
    return [label_alpha.strip(), label_split_numerics_alpha.strip(),label_split_numerics_alpha_camel.strip()]

def process_label(label):
    return re.sub('\s+', ' ', label)

,from,label
0,http://purl.obolibrary.org/obo/GECKO_0000071,sleep
1,http://purl.obolibrary.org/obo/GECKO_0000094,associated disease(s)
2,http://purl.obolibrary.org/obo/OBI_0002118,WES
3,http://purl.obolibrary.org/obo/OBI_0000537,"Sequence variants (CNV, SNP arrays)"
4,http://purl.obolibrary.org/obo/GECKO_0000010,sex(es) studied in cohort
...,...,...
83,http://purl.obolibrary.org/obo/OBI_0000435,DNA/Genotyping
84,http://purl.obolibrary.org/obo/GECKO_0000067,lifestyle and behaviours
85,http://purl.obolibrary.org/obo/UBERON_0001088,urine
86,http://purl.obolibrary.org/obo/CMO_0000294,heart rate (HR)


In [128]:
data=pd.read_csv(args.file_simplestring,sep="\t")
data

,STUDY,BIOENTITY,PROPERTY_TYPE,PROPERTY_VALUE,SEMANTIC_TAG,ANNOTATOR,ANNOTATION_DATE
0,IHCC Simple String,https://purl.ihccglobal.org/GCS_FoodWatOthVeg,information_entity,FoodWatOthVeg,http://purl.obolibrary.org/obo/PATO_0001995,Knocean,2020-08-27 20:37:04
1,IHCC Simple String,https://purl.ihccglobal.org/GCS_FoodWatOthVeg,information_entity,Food Wat Oth Veg,http://purl.obolibrary.org/obo/PATO_0001995,Knocean,2020-08-27 20:37:04
2,IHCC Simple String,https://purl.ihccglobal.org/GCS_Vitamin_B12,information_entity,Vitamin B12,http://purl.obolibrary.org/obo/GECKO_0000072,Knocean,2020-08-27 20:37:04
3,IHCC Simple String,https://purl.ihccglobal.org/GCS_Vitamin_B12,information_entity,Vitamin B 12,http://purl.obolibrary.org/obo/GECKO_0000072,Knocean,2020-08-27 20:37:04
4,IHCC Simple String,https://purl.ihccglobal.org/GCS_TransfusionFlag,information_entity,TransfusionFlag,http://purl.obolibrary.org/obo/GECKO_0000091,Knocean,2020-08-27 20:37:04
...,...,...,...,...,...,...,...
4470,IHCC Simple String,https://purl.ihccglobal.org/VZ_FUGlucose,information_entity,Follow Up Glucose,http://purl.obolibrary.org/obo/UBERON_0000178,Knocean,2020-08-27 20:37:04
4471,IHCC Simple String,https://purl.ihccglobal.org/VZ_EDT2Collected,information_entity,EDT2 Collected,http://purl.obolibrary.org/obo/GECKO_0000112,Knocean,2020-08-27 20:37:04
4472,IHCC Simple String,https://purl.ihccglobal.org/VZ_EDT2Collected,information_entity,EDT 2 Collected,http://purl.obolibrary.org/obo/GECKO_0000112,Knocean,2020-08-27 20:37:04
4473,IHCC Simple String,https://purl.ihccglobal.org/VZ_RadiologistLung...,information_entity,Radiologist LungFields,http://purl.obolibrary.org/obo/CMO_0000000,Knocean,2020-08-27 20:37:04


In [131]:
gecko=pd.read_csv(args.gecko_path,sep=",")
gecko_labels=gecko[gecko['property']=="http://www.w3.org/2000/01/rdf-schema#label"][['from','label']]
gecko=gecko[['from','label']]
gecko

,from,label
0,http://purl.obolibrary.org/obo/GECKO_0000071,sleep
1,http://purl.obolibrary.org/obo/GECKO_0000094,associated disease(s)
2,http://purl.obolibrary.org/obo/OBI_0002118,WES
3,http://purl.obolibrary.org/obo/OBI_0000537,"Sequence variants (CNV, SNP arrays)"
4,http://purl.obolibrary.org/obo/GECKO_0000010,sex(es) studied in cohort
...,...,...
83,http://purl.obolibrary.org/obo/OBI_0000435,DNA/Genotyping
84,http://purl.obolibrary.org/obo/GECKO_0000067,lifestyle and behaviours
85,http://purl.obolibrary.org/obo/UBERON_0001088,urine
86,http://purl.obolibrary.org/obo/CMO_0000294,heart rate (HR)


In [148]:
X = data.PROPERTY_VALUE
X = X.str.replace('[\s]+',' ')
y = data.SEMANTIC_TAG.tolist()
print(X[:4])
print(y[:4])

0       FoodWatOthVeg
1    Food Wat Oth Veg
2         Vitamin B12
3        Vitamin B 12
Name: PROPERTY_VALUE, dtype: object
['http://purl.obolibrary.org/obo/PATO_0001995', 'http://purl.obolibrary.org/obo/PATO_0001995', 'http://purl.obolibrary.org/obo/GECKO_0000072', 'http://purl.obolibrary.org/obo/GECKO_0000072']


In [150]:
# Modification 1, adding GECKO metadata to augment training data. 
X=X.append(gecko.label.str.replace('[\s]+',' '))
y.extend(gecko['from'].tolist())
print(X[:4])
print(y[:4])

0       FoodWatOthVeg
1    Food Wat Oth Veg
2         Vitamin B12
3        Vitamin B 12
dtype: object
['http://purl.obolibrary.org/obo/PATO_0001995', 'http://purl.obolibrary.org/obo/PATO_0001995', 'http://purl.obolibrary.org/obo/GECKO_0000072', 'http://purl.obolibrary.org/obo/GECKO_0000072']


In [151]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100) 
# I removed , stratify = y

In [152]:
y_test[:5]

['http://purl.obolibrary.org/obo/STATO_0000093',
 'http://purl.obolibrary.org/obo/GECKO_0000064',
 'http://purl.obolibrary.org/obo/UBERON_0000178',
 'http://purl.obolibrary.org/obo/GECKO_0000068',
 'http://purl.obolibrary.org/obo/GECKO_0000064']

In [153]:
count_vect = CountVectorizer(ngram_range = (1,2),min_df=2).fit(X_train)
X_train_counts = count_vect.transform(X_train)

In [154]:
X_train_counts.shape

(3720, 2132)

In [155]:
vocabulary = count_vect.vocabulary_

In [156]:
#Get the most frequent words
sum_words = X_train_counts.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vocabulary.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
print(words_freq[:20])

[('of', 184), ('mhd', 174), ('score', 131), ('and', 130), ('date', 118), ('code', 105), ('q1', 104), ('q2', 98), ('age', 69), ('medical', 64), ('officer', 61), ('medical officer', 61), ('to', 61), ('diagnostic', 58), ('cause', 53), ('or', 52), ('multiple', 51), ('cause code', 50), ('vv', 49), ('multiple cause', 49)]


In [157]:
print(words_freq[-10:])

[('q1 kr12', 2), ('q1 kr4', 2), ('birthplace', 2), ('to non', 2), ('non existing', 2), ('reticularopacities', 2), ('lung reticularopacities', 2), ('age band', 2), ('diagnostic test', 2), ('on admission', 2)]


In [158]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.1).fit(X_train_counts, y_train)

In [159]:
from sklearn.metrics import accuracy_score
X_test_counts = count_vect.transform(X_test)
y_pred  = clf.predict(X_test_counts)

In [160]:
X_test_counts

<931x2132 sparse matrix of type '<class 'numpy.int64'>'
	with 2673 stored elements in Compressed Sparse Row format>

In [161]:
def print_results(y_test,y_pred):
    acc = accuracy_score(y_test,y_pred)
    print('Overall accuracy : '+str(acc))
    f1_micro = f1_score(y_test, y_pred, average = 'micro')
    f1_macro = f1_score(y_test, y_pred, average = 'macro')
    f1_all = f1_score(y_test, y_pred, average = None)

    print('F1 micro : ' +str(f1_micro))
    print('F1 macro : ' +str(f1_macro))
    print('F1 for each class : ' +str(f1_all))

In [162]:
print_results(y_test,y_pred)

Overall accuracy : 0.640171858216971
F1 micro : 0.640171858216971
F1 macro : 0.38200407178865897
F1 for each class : [0.65217391 0.         0.83333333 0.         0.66666667 0.
 1.         0.         0.         0.         0.73684211 0.
 0.         0.62068966 0.         0.         0.         0.58653846
 0.16666667 0.         0.8        0.8        0.         0.75
 0.         0.6        0.         0.7804878  0.58823529 0.
 0.71428571 0.4        0.55555556 0.82758621 0.28571429 1.
 0.64516129 0.57142857 0.         0.5        0.         0.71641791
 0.75675676 0.         0.71428571 0.66666667 1.         0.33333333
 0.85714286 0.54545455 1.         0.         0.         0.
 0.84705882 0.66666667 0.         0.67647059 0.21428571 0.
 0.63636364 0.         0.5        0.         0.         0.        ]


###### TFIDF

In [163]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(ngram_range = (1,2), min_df = 2).fit(X_train)
X_train_tfidf = tfidf_vect.transform(X_train)
X_train_tfidf

<3720x2132 sparse matrix of type '<class 'numpy.float64'>'
	with 12143 stored elements in Compressed Sparse Row format>

In [164]:
y_train[:5]

['http://purl.obolibrary.org/obo/OGMS_0000020',
 'http://purl.obolibrary.org/obo/PATO_0000011',
 'http://purl.obolibrary.org/obo/PATO_0000011',
 'http://purl.obolibrary.org/obo/GECKO_0000114',
 'http://purl.obolibrary.org/obo/MONDO_0005151']

In [165]:
clf2 = MultinomialNB(alpha = 0.1).fit(X_train_tfidf, y_train)
X_test_tfidf = tfidf_vect.transform(X_test)
y_pred  = clf2.predict(X_test_tfidf)
acc = accuracy_score(y_test,y_pred)

In [166]:
print_results(y_test,y_pred)

Overall accuracy : 0.6433941997851772
F1 micro : 0.6433941997851772
F1 macro : 0.35760086012629066
F1 for each class : [0.68085106 0.83333333 0.         0.66666667 0.         1.
 0.         0.         0.         0.75675676 0.         0.
 0.41666667 0.         0.         0.         0.59509202 0.
 0.         0.8        1.         0.         0.8        0.
 0.6        0.         0.7804878  0.625      0.         0.76923077
 0.4        0.55555556 0.82758621 0.         0.         0.62068966
 0.57142857 0.         0.53333333 0.         0.70769231 0.73504274
 0.72       0.66666667 0.8        0.4        0.         0.57142857
 0.66666667 0.         0.         0.         0.83236994 0.73684211
 0.         0.69565217 0.26086957 0.         0.64516129 0.
 0.61538462 0.         0.         0.        ]


###### SVM classifier (linear with stochastic gradient descent for faster computation)

In [167]:
from sklearn.linear_model import SGDClassifier
clf_svg = SGDClassifier(loss='hinge', penalty='l2', alpha = 1e-05).fit(X_train_tfidf,y_train)
y_pred_svg  = clf_svg.predict(X_test_tfidf)

In [168]:
print_results(y_test,y_pred_svg)

Overall accuracy : 0.6638023630504833
F1 micro : 0.6638023630504833
F1 macro : 0.5759587524374079
F1 for each class : [0.69387755 0.         0.92307692 0.57142857 0.5        0.
 1.         0.25       1.         0.         0.7        1.
 1.         0.38095238 1.         1.         0.         0.60128617
 0.5        0.66666667 1.         0.8        1.         0.75362319
 0.66666667 0.6        0.         0.82051282 0.70588235 0.
 0.625      0.66666667 0.57894737 0.8        0.         1.
 0.6        0.57142857 0.         0.8        0.         0.66666667
 0.78571429 0.66666667 0.77777778 1.         0.28571429 0.4
 0.57142857 1.         0.         1.         0.         0.88343558
 0.77777778 1.         0.65151515 0.26086957 0.5        0.65168539
 0.66666667 0.61538462 0.5        0.         0.        ]


In [169]:
#check accuracy in training set to do a sanity check for overfitting
pred_train = clf_svg.predict(X_train_tfidf)
acc_train = accuracy_score(y_train,pred_train)
print(acc_train)

0.7741935483870968


###### Stemming

In [170]:
from gensim.parsing.porter import PorterStemmer

p = PorterStemmer()

for i in range(len(X)):
    X.iloc[i] = p.stem_sentence(X.iloc[i])
print(X.head())

0       foodwatothveg
1    food wat oth veg
2         vitamin b12
3        vitamin b 12
4     transfusionflag
dtype: object


In [171]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=100)
count_vectorize = CountVectorizer(ngram_range = (1,2), stop_words = 'english', min_df = 2)

tfidf_transform = TfidfTransformer()

pipeline_svm = Pipeline([('bow', count_vectorize),('tfidf', tfidf_transform),('clf_svg', 
                              SGDClassifier(loss='hinge', penalty='l2', alpha = 1e-05))])
#pipeline_svm = Pipeline([('bow', count_vectorize),('tfidf', tfidf_transform),('clf_svm', SVC(kernel = 'rbf'))])

clf_stem = pipeline_svm.fit(X_train, y_train)
y_pred_stem  = clf_stem.predict(X_test)

In [172]:
print_results(y_test,y_pred_stem)

Overall accuracy : 0.6523605150214592
F1 micro : 0.6523605150214592
F1 macro : 0.6260155098344165
F1 for each class : [0.66666667 1.         1.         0.66666667 1.         0.
 1.         0.66666667 0.82352941 1.         0.25       0.76190476
 0.         1.         1.         0.80952381 0.         0.66666667
 0.8        0.85714286 0.93333333 0.         0.75       0.
 0.6        0.71428571 1.         1.         0.13432836 0.75
 1.         0.68965517 0.74626866 0.54545455 1.         0.8
 0.         0.66666667 0.         1.         0.         0.925
 0.92307692 1.         0.69565217 0.22222222 0.66666667 0.57142857
 0.5        0.75       0.         0.        ]


# Validating against test data

In [173]:
tsv=pd.read_csv(args.tsv_path,sep=",")
template=pd.read_csv(args.template_path,sep="\t")

template['IRI']=["https://purl.ihccglobal.org/"+str(item).replace(":","_") for item in template['Term ID']]
del template['Suggested Categories']
template=pd.merge(template, tsv[['from','to']], how='left', left_on=['IRI'], right_on=['from'])
template

,Term ID,Label,Parent Term,Definition,GECKO Category,Comment,IRI,from,to
0,ID,LABEL,C % SPLIT=|,A definition,NaN,NaN,https://purl.ihccglobal.org/ID,NaN,NaN
1,NaN,is-required;,NaN,NaN,NaN,NaN,https://purl.ihccglobal.org/nan,NaN,NaN
2,KoGES:0000001,Core Variables,NaN,NaN,NaN,NaN,https://purl.ihccglobal.org/KoGES_0000001,NaN,NaN
3,KoGES:0000002,Core Questionnaires,KoGES:0000001,NaN,questionnaire/survey data,NaN,https://purl.ihccglobal.org/KoGES_0000002,https://purl.ihccglobal.org/KoGES_0000002,GECKO:0000056
4,KoGES:0000003,Socio-demographic data,KoGES:0000002,NaN,socio-demographic and economic characteristics,NaN,https://purl.ihccglobal.org/KoGES_0000003,https://purl.ihccglobal.org/KoGES_0000003,PATO:0001995
...,...,...,...,...,...,...,...,...,...
204,KoGES:0000204,Calcium (Urine),KoGES:0000187,NaN,urine,NaN,https://purl.ihccglobal.org/KoGES_0000204,https://purl.ihccglobal.org/KoGES_0000204,UBERON:0001088
205,KoGES:0000205,Sodium (Urine),KoGES:0000187,NaN,urine,NaN,https://purl.ihccglobal.org/KoGES_0000205,https://purl.ihccglobal.org/KoGES_0000205,UBERON:0001088
206,KoGES:0000206,Potassium (Urine),KoGES:0000187,NaN,urine,NaN,https://purl.ihccglobal.org/KoGES_0000206,https://purl.ihccglobal.org/KoGES_0000206,UBERON:0001088
207,KoGES:0000207,Vitamin C,KoGES:0000187,NaN,urine,no CMO term for vit C levels in urine,https://purl.ihccglobal.org/KoGES_0000207,https://purl.ihccglobal.org/KoGES_0000207,UBERON:0001088


In [174]:
tx=template[['Label','to']].copy()
tx['to']=[obo_id_to_iri(item) for item in template['to']]
tx=tx.dropna()
X_validate=tx['Label'].values
y_validate=tx['to'].values

tx

,Label,to
3,Core Questionnaires,http://purl.obolibrary.org/obo/GECKO_0000056
4,Socio-demographic data,http://purl.obolibrary.org/obo/PATO_0001995
5,Education level,http://purl.obolibrary.org/obo/GECKO_0000065
6,Marital status,http://purl.obolibrary.org/obo/PATO_0001995
7,Household income,http://purl.obolibrary.org/obo/PATO_0001995
...,...,...
204,Calcium (Urine),http://purl.obolibrary.org/obo/UBERON_0001088
205,Sodium (Urine),http://purl.obolibrary.org/obo/UBERON_0001088
206,Potassium (Urine),http://purl.obolibrary.org/obo/UBERON_0001088
207,Vitamin C,http://purl.obolibrary.org/obo/UBERON_0001088


In [175]:
tfidf_validate = tfidf_vect.transform(X_validate)
pred_validate  = clf_svg.predict(tfidf_validate)
print_results(y_validate,pred_validate)

Overall accuracy : 0.225
F1 micro : 0.225
F1 macro : 0.22128044863135013
F1 for each class : [0.26666667 0.66666667 0.         0.8        0.66666667 0.
 0.         0.         0.         1.         0.         0.
 0.54545455 0.         0.28571429 0.         0.         1.
 0.         0.         0.57142857 0.18181818 0.         0.
 0.         0.         0.         0.         0.         0.4
 0.         0.66666667 0.         0.33009709 0.36363636]


In [176]:
acc = accuracy_score(koges_test,koges_pred)
pred_validate[:3]

array(['http://purl.obolibrary.org/obo/GECKO_0000052',
       'http://purl.obolibrary.org/obo/PATO_0001995',
       'http://purl.obolibrary.org/obo/GECKO_0000065'], dtype='<U45')

In [177]:

df_out = pd.DataFrame(
    {'Label': X_validate,
     'Suggested Categories': pred_validate
    })
df_out=pd.merge(df_out, gecko_labels, how='left', left_on=['Suggested Categories'], right_on=['from'])
df_out['Suggested Categories']=[str(item).replace(obo_purl,"").replace("_",":") if item.startswith(obo_purl) else item for item in df_out['Suggested Categories']]
df_out['Suggested Categories']="MEDIUM "+df_out['Suggested Categories']+" "+df_out['label']
del df_out['from']
del df_out['label']
df_out

,Label,Suggested Categories
0,Core Questionnaires,MEDIUM GECKO:0000052 survey administration
1,Socio-demographic data,MEDIUM PATO:0001995 socio-demographic and econ...
2,Education level,MEDIUM GECKO:0000065 education
3,Marital status,MEDIUM PATO:0001995 socio-demographic and econ...
4,Household income,MEDIUM PATO:0001995 socio-demographic and econ...
...,...,...
195,Calcium (Urine),MEDIUM UBERON:0001088 urine
196,Sodium (Urine),MEDIUM UBERON:0001088 urine
197,Potassium (Urine),MEDIUM UBERON:0001088 urine
198,Vitamin C,MEDIUM GECKO:0000072 nutrition


In [178]:
template_out=pd.merge(template, df_out.drop_duplicates(), how='left', left_on=['Label'], right_on=['Label'])
template_out=template_out[['Term ID','Label','Parent Term','Definition','Comment','Suggested Categories']]
template_out=template_out.drop_duplicates()
template_out

,Term ID,Label,Parent Term,Definition,Comment,Suggested Categories
0,ID,LABEL,C % SPLIT=|,A definition,NaN,NaN
1,NaN,is-required;,NaN,NaN,NaN,NaN
2,KoGES:0000001,Core Variables,NaN,NaN,NaN,NaN
3,KoGES:0000002,Core Questionnaires,KoGES:0000001,NaN,NaN,MEDIUM GECKO:0000052 survey administration
4,KoGES:0000003,Socio-demographic data,KoGES:0000002,NaN,NaN,MEDIUM PATO:0001995 socio-demographic and econ...
...,...,...,...,...,...,...
204,KoGES:0000204,Calcium (Urine),KoGES:0000187,NaN,NaN,MEDIUM UBERON:0001088 urine
205,KoGES:0000205,Sodium (Urine),KoGES:0000187,NaN,NaN,MEDIUM UBERON:0001088 urine
206,KoGES:0000206,Potassium (Urine),KoGES:0000187,NaN,NaN,MEDIUM UBERON:0001088 urine
207,KoGES:0000207,Vitamin C,KoGES:0000187,NaN,no CMO term for vit C levels in urine,MEDIUM GECKO:0000072 nutrition


In [179]:
with open(args.tsv_out_path,'w') as write_csv:
    write_csv.write(template_out.to_csv(sep='\t', index=False))